<a href="https://colab.research.google.com/github/Dohy-Lee/NLP_By_Pytorch/blob/main/ch4_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 피드 포워드 신경망
### 데이터가 한 방향으로 (입력 → 출력) 흐르는 신경망. 정의에 따르면, 퍼셉트론도 피드 포워드 신경망지만 보통 유닛이 여럿인 복잡한 모델에 사용함
* 다층 퍼셉트론(MLP) : 많은 퍼셉트론이 있는 층을 여러 개 쌓아 올린 구조  
 * 입력 : 데이터 벡터
 * 출력 : 벡터
 * 층 사이에 비선형성(활성화 함수)을 추가할 수 있음 → 선형적으로 구분할 수 있음(XOR 분류 문제 해결)
 * 최종 은닉 벡터는 항상 Linear 층과 비선형 함수를 사용하여 출력 벡터에 매핑 
* 합성곱 신경망(CNN) 


In [30]:
#파이토치를 사용한 MLP
import torch.nn as nn
import torch.nn.functional as F
import torch
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim): # input_dim(int) : 입력 벡터 크기, hidden_dim(int) : 첫 번째 Linear 층의 출력 크기, 두 번째 Linear 층의 출력 크기
      super(MultilayerPerceptron, self).__init__()
      self.fc1 = nn.Linear(input_dim, hidden_dim)
      self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False): # MLP 정방향 계산 
                                                  # x_in(torch.Tensor) : 입력 데이터 텐서, x_in.shape는 (batch, input_dim), apply_softmax(bool) : 소프트맥스 활성화 함수 플래그. 크로스 엔트로피 손실을 사용하려면 False
                                                  # 반환 : 결과 텐서. tensor.shape는 (batch, output_dim)
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)

        if apply_softmax:
          output = F.softmax(output, dim=1)
        return output

In [31]:
 # 입력 차원의 크기 3, 출력 차원의 크기 4, 은닉 차원의 크기 100
 batch_size = 2 # 한 번에 입력할 샘플 개수
 input_dim = 3
 hidden_dim = 100
 output_dim = 4

 # MLP 모델 생성
 mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
 print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


In [32]:
# 랜덤한 입력으로 MLP 테스트하기
def describe(x):
  print("타입 : {}".format(x.type))
  print('크기 : {}'.format(x.shape))
  print('값 : {}'.format(x))

x_input = torch.rand(batch_size, input_dim)
describe(x_input)

타입 : <built-in method type of Tensor object at 0x7fa9beeee7d0>
크기 : torch.Size([2, 3])
값 : tensor([[0.5951, 0.5624, 0.9027],
        [0.0888, 0.6399, 0.8904]])


In [33]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입 : <built-in method type of Tensor object at 0x7fa9beeeebf0>
크기 : torch.Size([2, 4])
값 : tensor([[ 0.1959,  0.2301,  0.2000, -0.1604],
        [ 0.1395,  0.1968,  0.1551, -0.1158]], grad_fn=<AddmmBackward0>)


출력 2x4 행렬  
이 텐서의 행은 배치 차원에 해당 = 미니배치의 데이터 포인트 개수  
열은 각 데이터 포인트에 대한 최종 특성 벡터(표현벡터). 분류 등에서는 특성 벡터 = 예측 벡터  
 * 예측 벡터 : 확률 분포에 대응. 훈련을 수행하는지 추론을 수행하는지에 따라 예측 벡터로 하는 일이 다름
  * 훈련에서는 출력을 그대로 손실 함수 및 타깃 클래스 레이블과 함께 사용
  * 확률로 바꾸기 위해선 소프트맥스 활성화 함수(큰 양숫값이 높은 확률을 만들고, 낮은 음숫값은 작은 확률을 만듦)가 필요함  
  파이토치에서는 apply_softmax 매개변수를 사용해서 이 단계를 적용할 수 있음


In [34]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입 : <built-in method type of Tensor object at 0x7fa9beeeedd0>
크기 : torch.Size([2, 4])
값 : tensor([[0.2674, 0.2767, 0.2685, 0.1873],
        [0.2598, 0.2751, 0.2639, 0.2013]], grad_fn=<SoftmaxBackward0>)


## MLP로 성씨 분류하기
* 데이터 분류

In [35]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [36]:
args = Namespace(
    raw_dataset_csv="surnames.csv",
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="surnames_with_splits.csv",
    seed=1337
)


In [37]:
surnames = pd.read_csv(args.raw_dataset_csv, header=0)
surnames.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


In [38]:
set(surnames.nationality) #성씨에 대한 나라 정보

{'Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese'}

In [39]:
# 나라별로 데이터 분리
by_nationality = collections.defaultdict(list)
for _, row in surnames.iterrows():
    by_nationality[row.nationality].append(row.to_dict())
final_list=[]
np.random.seed(args.seed)
for _, item_list in sorted(by_nationality.items()):
    np.random.shuffle(item_list) # 원본 데이터의 70% 이상이 러시아 이름이라 편중되어 있어서 셔플링
    n = len(item_list)
    n_train = int(args.train_proportion*n)
    n_val = int(args.val_proportion*n)
    n_test = int(args.test_proportion*n)
    
    # item은 dict 자료형, key로 'split'속성 부여
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  
    
    final_list.extend(item_list)

In [40]:
final_surnames = pd.DataFrame(final_list)
final_surnames.split.value_counts()

train    7680
test     1660
val      1640
Name: split, dtype: int64

In [41]:
final_surnames.head()

,surname,nationality,split
0,Totah,Arabic,train
1,Abboud,Arabic,train
2,Fakhoury,Arabic,train
3,Srour,Arabic,train
4,Sayegh,Arabic,train


In [42]:
final_surnames.to_csv(args.output_munged_csv, index=False) #CSV로 저장

# MLP로 분류  
Vocabulary, Vectorizer, DataLoader로 성씨 문자열을 벡터의 미니배치로 변환  
문자를 정수에 매핑하는 식으로 데이터를 벡터로 반환

In [43]:
from argparse import Namespace
from collections import Counter
import json
import os
import string

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

Vocabulary

In [44]:
class Vocabulary(object): #매핑을 위해 텍스트를 처리하고, 어휘 사전을 만드는 클래스

    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"): # token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
                                                                            # add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
                                                                            # unk_token (str): Vocabulary에 추가할 UNK 토큰

        if token_to_idx is None: # 토큰과 정수 간의 상호 변환을 위해 딕셔너리 2개 생성
            token_to_idx = {}

        self._token_to_idx = token_to_idx # {"Token" : "idx"}
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()} # {"idx" : "Token"}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token) 
        
        
    def to_serializable(self): # 직렬화할 수 있는 딕셔너리 반환
        return {'token_to_idx': self._token_to_idx, 
                'add_unk': self._add_unk, 
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents): # 직렬화된 딕셔너리에서 Vocabulary 객체 만듦
        return cls(**contents)

    def add_token(self, token): # 토큰을 기반으로 매핑 딕셔너리 업데이트
                                # token (str): Vocabulary에 추가할 토큰
                                # 반환값 : index (int): 토큰에 상응하는 정수
        try:
            index = self._token_to_idx[token]
        except KeyError:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens): # 토큰리스트를 Vocabulary에 추가
                                # tokens (list): 문자열 토큰 리스트
                                # 반환값 : indices (list): 토큰 리스트에 상응되는 인덱스 리스트
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token): # 주어진 토큰에 해당하는 인덱스 반환, 토큰이 없다면 UNK 인덱스 반환.
                                   # token (str): 찾을 토큰 
                                   # 반환값 : index (int): 토큰에 해당하는 인덱스
                                   # UNK 토큰을 사용하려면(Vocabulary에 추가하기 위해서) 'unk_index'가 0 보다 커야함
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index): # 주어진 인덱스에 해당하는 토큰 반환
                                   # index (int): 찾을 인덱스
                                   # 반환값 : token (str): 인덱스에 해당하는 토큰

        if index not in self._idx_to_token: # Vocabulary에 없을 때 발생
            raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

Vectorizer

In [45]:
class SurnameVectorizer(object): # Vocabulary를 적용해 성씨 문자열을 벡터로 바꿈
    def __init__(self, surname_vocab, nationality_vocab): # surname_vocab (Vocabulary): 문자를 정수에 매핑하는 Vocabulary 객체
                                                          # nationality_vocab (Vocabulary): 국적을 정수에 매핑하는 Vocabulary 객체
        self.surname_vocab = surname_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname): # 성씨에 대한 원-핫 벡터 생성
                                  # surname (str): 성씨
                                  # 반환값 : one_hot(np.ndarray): 원-핫 벡터
        vocab = self.surname_vocab
        one_hot = np.zeros(len(vocab), dtype=np.float32)
        for token in surname:
            one_hot[vocab.lookup_token(token)] = 1

        return one_hot

    @classmethod
    def from_dataframe(cls, surname_df): # 데이터셋 데이터프레임에서 Vectorizer 객체 생성
                                         # surname_df (pandas.DataFrame): 성씨 데이터셋
                                         # 반환값 : SurnameVectorizer 객체
        surname_vocab = Vocabulary(unk_token="@")
        nationality_vocab = Vocabulary(add_unk=False)

        for index, row in surname_df.iterrows():
            for letter in row.surname:
                surname_vocab.add_token(letter)
            nationality_vocab.add_token(row.nationality)

        return cls(surname_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents): # 직렬화된 딕셔너리에서 Vocabulary 생성
        surname_vocab = Vocabulary.from_serializable(contents['surname_vocab'])
        nationality_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])
        return cls(surname_vocab=surname_vocab, nationality_vocab=nationality_vocab)

    def to_serializable(self): # 직렬화할수 있는 딕셔너리 생성
        return {'surname_vocab': self.surname_vocab.to_serializable(),
                'nationality_vocab': self.nationality_vocab.to_serializable()}

Dataset

In [46]:
class SurnameDataset(Dataset): # __getitem__, __len__ 구현必
    def __init__(self, surname_df, vectorizer): # surname_df (pandas.DataFrame): 데이터셋
                                                # vectorizer (SurnameVectorizer): SurnameVectorizer 객체
    
        self.surname_df = surname_df
        self._vectorizer = vectorizer

        self.train_df = self.surname_df[self.surname_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.surname_df[self.surname_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.surname_df[self.surname_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')
        
        # 클래스 가중치
        class_counts = surname_df.nationality.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.nationality_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, surname_csv): # 데이터셋을 로드하고 새로운 SurnameVectorizer 객체 생성
                                                            # review_csv (str): 데이터셋의 위치
                                                            # SurnameDataset의 인스턴스
        surname_df = pd.read_csv(surname_csv)
        train_surname_df = surname_df[surname_df.split=='train']
        return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath): # 데이터셋을 로드하고 새로운 SurnameVectorizer 객체를 생성
                                                                                 # 캐시된 SurnameVectorizer 객체를 재사용할 때 사용
                                                                                 # surname_csv (str): 데이터셋의 위치
                                                                                 # vectorizer_filepath (str): SurnameVectorizer 객체의 저장 위치
                                                                                 # 반환값 : SurnameDataset의 인스턴스
        surname_df = pd.read_csv(surname_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(surname_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath): # 파일에서 SurnameVectorizer 객체를 로드하는 정적 메서드
                                                   # vectorizer_filepath (str): 직렬화된 SurnameVectorizer 객체의 위치
                                                   # 반환값 : SurnameVectorizer의 인스턴스
        with open(vectorizer_filepath) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath): # SurnameVectorizer 객체를 json 형태로 디스크에 저장
                                                    # vectorizer_filepath (str): SurnameVectorizer 객체의 저장 위치
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self): # 벡터 변환 객체 반환
        return self._vectorizer

    def set_split(self, split="train"): # 분할 세트 
                                        # split (str): "train", "val", "test" 중 하나
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self): # 데이터셋의 길이 반환
        return self._target_size

    def __getitem__(self, index): # 원래 __getitem__ : 주어진 인덱스의 데이터 포인트를 반환.  여기선 벡터로 바꾼 성씨와 국적에 해당하는 인덱스 반환
                                  # index (int): 데이터 포인트의 인덱스
                                  # 반환값 : 데이터 포인트의 특성(x_surname)과 레이블(y_nationality)로 이루어진 딕셔너리
        row = self._target_df.iloc[index]

        surname_vector = \
            self._vectorizer.vectorize(row.surname)

        nationality_index = \
            self._vectorizer.nationality_vocab.lookup_token(row.nationality)

        return {'x_surname': surname_vector,
                'y_nationality': nationality_index}

    def get_num_batches(self, batch_size): # 주어진 배치 크기를 데이터셋으로 만들 수 있는 배치 개수 반환
                                           # batch_size (int) : 배치 크기
                                           # 반환값 : 배치 개수
        return len(self) // batch_size

    
def generate_batches(dataset, batch_size, shuffle=True, # 파이토치의 DataLoader를 감싸고 있는 제너레이터 함수
                     drop_last=True, device="cpu"): # 각 텐서를 지정된 장치로 이동
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict


Model

In [47]:

class SurnameClassifier(nn.Module): # 성씨 분류를 위한 MLP. 
                                    # 첫 번쨰 Linear 층이 입력 벡터를 중간 벡터로 매핑하고 이 벡터에 비선형 활성화 함수를 적용, 두 번쨰 Linear 층이 중간 벡터를 예측 벡터로 매핑 
                                    # 마지막 단계에서 출력의 합을 1로 만드는 데 소프트맥스 함수를 선택적으로 적용 → 확률로 해석할 수 있음
    def __init__(self, input_dim, hidden_dim, output_dim): # input_dim (int): 입력 벡터 크기
                                                           # hidden_dim (int): 첫 번째 Linear 층의 출력 크기
                                                           # output_dim (int): 두 번째 Linear 층의 출력 크기
        super(SurnameClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False): # MLP 정방향 계산
                                                  # x_in (torch.Tensor): 입력 데이터 텐서, x_in.shape는 (batch, input_dim)
                                                  # apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그, 크로스-엔트로피 손실을 사용하려면 False로 지정
                                                  # 반환값: 결과 텐서. tensor.shape은 (batch, output_dim)입니다

        intermediate_vector = F.relu(self.fc1(x_in))
        prediction_vector = self.fc2(intermediate_vector)
        #output = self.fc2(F.dropout(intermediate, p=0.5)) #드롭 아웃 설정
        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)
        return prediction_vector

Train

In [48]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state): # 훈련 상태를 업데이트
                                                  # args: 메인 매개변수, model: 훈련할 모델, train_state: 훈련 상태를 담은 딕셔너리
                                                  # 반환값: 새로운 훈련 상태
    # 적어도 한 번 모델을 저장
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # 성능이 향상되면 모델을 저장
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # 손실이 나빠지면
        if loss_t >= train_state['early_stopping_best_val']:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
        # 손실이 감소하면
        else:
            # 최상의 모델 저장
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0

        # 조기 종료 여부 확인
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

Utility

In [49]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

Setting

In [50]:
args = Namespace(
    # 날짜와 경로 정보
    surname_csv="surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch4/surname_mlp",
    # 모델 하이퍼파라미터
    hidden_dim=300,
    # 훈련 하이퍼파라미터
    seed=1337,
    num_epochs=100,
    early_stopping_criteria=5,
    learning_rate=0.001,
    batch_size=64,
    # 실행 옵션
    cuda=False,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("파일 경로: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
    
print("CUDA 사용여부: {}".format(args.cuda))

# 재현성을 위해 시드 설정
set_seed_everywhere(args.seed, args.cuda)

# 디렉토리 처리
handle_dirs(args.save_dir)

파일 경로: 
	model_storage/ch4/surname_mlp/vectorizer.json
	model_storage/ch4/surname_mlp/model.pth
CUDA 사용여부: False


In [51]:
if args.reload_from_files:
    # 체크포인트에서 훈련을 다시 시작
    print("로딩!")
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                              args.vectorizer_file)
else:
    # 데이터셋과 Vectorizer 만들기
    print("새로 만들기!")
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)
    
vectorizer = dataset.get_vectorizer()
classifier = SurnameClassifier(input_dim=len(vectorizer.surname_vocab), 
                               hidden_dim=args.hidden_dim, 
                               output_dim=len(vectorizer.nationality_vocab))


새로 만들기!


In [52]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm.notebook.tqdm(desc='training routine', 
                               total=args.num_epochs,
                               position=0)

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                               total=dataset.get_num_batches(args.batch_size), 
                               position=1, 
                               leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                             total=dataset.get_num_batches(args.batch_size), 
                             position=1, 
                             leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # 훈련 세트에 대한 순회

        # 훈련 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 훈련 과정은 총 5단계

            # --------------------------------------
            # 단계 1. 그레이디언트를 0으로 초기화
            optimizer.zero_grad()

            # 단계 2. 출력을 계산
            y_pred = classifier(batch_dict['x_surname'])

            # 단계 3. 손실을 계산
            loss = loss_func(y_pred, batch_dict['y_nationality'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # 단계 4. 손실을 사용해 그레이디언트를 계산
            loss.backward()

            # 단계 5. 옵티마이저로 가중치를 업데이트
            optimizer.step()
            # -----------------------------------------

            # 정확도를 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # 진행 바 업데이트
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # 검증 세트에 대한 순회

        # 검증 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # 단계 1. 출력을 계산
            y_pred =  classifier(batch_dict['x_surname'])

            # 단계 2. 손실을 계산
            loss = loss_func(y_pred, batch_dict['y_nationality'])
            loss_t = loss.to("cpu").item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # 단계 3. 정확도를 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")


training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/120 [00:00<?, ?it/s]

split=val:   0%|          | 0/25 [00:00<?, ?it/s]

In [53]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산
classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)


dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력을 계산
    y_pred =  classifier(batch_dict['x_surname'])
    
    # 손실을 계산
    loss = loss_func(y_pred, batch_dict['y_nationality'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산
    acc_t = compute_accuracy(y_pred, batch_dict['y_nationality'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [54]:
print("테스트 손실: {};".format(train_state['test_loss']))
print("테스트 정확도: {}".format(train_state['test_acc']))

테스트 손실: 1.783166675567627;
테스트 정확도: 46.31249999999999


In [55]:
def predict_nationality(surname, classifier, vectorizer): # 새로운 성씨로 국적 예측
                                                          # surname (str): 분류할 성씨
                                                          # classifier (SurnameClassifer): 분류기 객체
                                                          # vectorizer (SurnameVectorizer): SurnameVectorizer 객체
                                                          # 반환값: 가장 가능성이 높은 국적과 확률로 구성된 딕셔너리

    vectorized_surname = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).view(1, -1)
    result = classifier(vectorized_surname, apply_softmax=True)

    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'nationality': predicted_nationality, 'probability': probability_value}

In [56]:
new_surname = input("분류하려는 성씨를 입력하세요: ")
classifier = classifier.to("cpu")
prediction = predict_nationality(new_surname, classifier, vectorizer)
print("{} -> {} (p={:0.2f})".format(new_surname,
                                    prediction['nationality'],
                                    prediction['probability']))

분류하려는 성씨를 입력하세요: kim
kim -> Polish (p=0.30)


In [57]:
vectorizer.nationality_vocab.lookup_index(8)

'Irish'

In [58]:
def predict_topk_nationality(name, classifier, vectorizer, k=5): # 새로운 성씨에 대한 최상위 K개 국적을 예측
                                                                 # surname (str): 분류하려는 성씨
                                                                 # classifier (SurnameClassifer): 분류기 객체
                                                                 # vectorizer (SurnameVectorizer): SurnameVectorizer 객체
                                                                 # k (int): the number of top nationalities to return
                                                                 # 반환값 : 딕셔너리 리스트, 각 딕셔너리는 국적과 확률로 구성
    vectorized_name = vectorizer.vectorize(name)
    vectorized_name = torch.tensor(vectorized_name).view(1, -1)
    prediction_vector = classifier(vectorized_name, apply_softmax=True)
    probability_values, indices = torch.topk(prediction_vector, k=k) # 예측값에서 Top K개 반환
    
    # 반환되는 크기는 (1,k)입니다
    probability_values = probability_values.detach().numpy()[0]
    indices = indices.detach().numpy()[0]
    
    results = []
    for prob_value, index in zip(probability_values, indices):
        nationality = vectorizer.nationality_vocab.lookup_index(index)
        results.append({'nationality': nationality, 
                        'probability': prob_value})
    
    return results


new_surname = input("분류하려는 성씨를 입력하세요: ")
classifier = classifier.to("cpu")

k = int(input("얼마나 많은 예측을 보고 싶나요? "))
if k > len(vectorizer.nationality_vocab):
    print("앗! 전체 국적 개수보다 큰 값을 입력했습니다. 모든 국적에 대한 예측을 반환합니다. :)")
    k = len(vectorizer.nationality_vocab)
    
predictions = predict_topk_nationality(new_surname, classifier, vectorizer, k=k)

print("최상위 {}개 예측:".format(k))
print("===================")
for prediction in predictions:
    print("{} -> {} (p={:0.2f})".format(new_surname,
                                        prediction['nationality'],
                                        prediction['probability']))


분류하려는 성씨를 입력하세요: kim
얼마나 많은 예측을 보고 싶나요? 50
앗! 전체 국적 개수보다 큰 값을 입력했습니다. 모든 국적에 대한 예측을 반환합니다. :)
최상위 18개 예측:
kim -> Polish (p=0.30)
kim -> Japanese (p=0.17)
kim -> Czech (p=0.17)
kim -> Korean (p=0.10)
kim -> Russian (p=0.10)
kim -> English (p=0.05)
kim -> German (p=0.04)
kim -> Dutch (p=0.02)
kim -> Scottish (p=0.02)
kim -> Arabic (p=0.01)
kim -> Greek (p=0.00)
kim -> Chinese (p=0.00)
kim -> Vietnamese (p=0.00)
kim -> Irish (p=0.00)
kim -> French (p=0.00)
kim -> Italian (p=0.00)
kim -> Spanish (p=0.00)
kim -> Portuguese (p=0.00)
